# Extração Automática e Agrupamento de Palavras-chave em Artigos Científicos Utilizando Mineração de Texto

Autor: Luiz Roberto Albano Junior <luiz.albano@unesp.br>

---

### Instalação de requisitos

**Alerta:** A instalação dos pacotes abaixo pode consumir cerca de 1Gb de espaço.

In [2]:
!pip install bertopic
!python -m spacy download pt_core_news_md


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 189, in _run_module_as_main
  File "<frozen runpy>", line 148, in _get_module_details
  File "<frozen runpy>", line 112, in _get_module_details
  File "/Users/luiz/Documents/Mestrado/Unesp/Mineracao/.venv/lib/python3.12/site-packages/spacy/__init__.py", line 6, in <module>
    from .errors import setup_default_warnings
  File "/Users/luiz/Documents/Mestrado/Unesp/Mineracao/.venv/lib/python3.12/site-packages/spacy/errors.py", line 3, in <module>
    from .compat import Literal
  File "/Users/lu

### Importação das bibliotecas necessárias

In [4]:
import sqlite3 as db
import pandas as pd
import spacy
from bertopic import BERTopic

/Users/luiz/Documents/Mestrado/Unesp/Mineracao/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Coleta de dados

A base de dados de artigos científicos foi obtida a partir da plataforma SciELO Brasil (https://www.scielo.br/). A escolha da plataforma deu-se após pesquisas de datasets contendo dados de artigos científicos. A excassez de um banco de dados confiável e que disponibilizasse o conteúdo destes artigos, fez com que pesquisasse serviços e formas de obter estas informações diretamente nas plataformas.

Após a análise de algumas plataformas, verifiquei a plataforma SciELO possibilitava realizar a coleta dos dados a partir de um script que realizou a "raspagem" dos dados. O download da base utilizada para este trabalho pode ser baixada em:<br>
https://drive.google.com/file/d/1LAfZSNz7AtfgZrbwKm2STH5FBvNBz2u-/view?usp=sharing
<br><br>
Formato: SQLite

In [5]:
conn = db.connect("dataset/scielo.papers.db")
cur = conn.cursor()

resultset = cur.execute("SELECT * FROM papers WHERE tokens IS NULL")
papers = resultset.fetchall()

## Limpeza e Pré-processamento de Texto (Tokenização)

In [6]:
pln = spacy.load("pt_core_news_md")

In [7]:
for paper in papers:
    document = pln(paper[7])
    
    tokens = []
    for token in document:
        if token.pos_ == "VERB" and token.is_alpha and not (token.is_stop) :
            tokens.append(str.lower(token.lemma_))
        elif not(token.pos_ == "VERB")  and token.is_alpha and not(token.is_stop) :
            tokens.append(str.lower(token.text))
    
    # Atualiza o registro na tabela
    cur.execute("UPDATE papers SET tokens = ? WHERE ID = ?", (" ".join(tokens), paper[0]))
    conn.commit()